# Introduction to Keras using the Breast Cancer Wisconsin (Diagnostic) Data Set
*Note: The dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29)*

## Project Outline

1. [Dataset description](#dataset-description)
2. [Load in data from the file _breast-cancer-wisconsin.data_](#load-in-data-from-the-file-_breast-cancer-wisconsin.data_)
3. Prune rows from dataset with errors
4. Format Dataset for use in Keras
5. Define network architecture
6. Train network on Data 
7. Visualize training

## 1. Dataset description

The data located in the file _breast-cancer-wisconsin.data_ is organized as follows: 

```
#  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class:                        (2 for benign, 4 for malignant)
```
More information on this dataset can be found in the _breast-cancer-wisconsin.names_ file.



## 2. Load in data from the file _breast-cancer-wisconsin.data_



In [7]:
import numpy as np

data = np.genfromtxt('breast-cancer-wisconsin.data', delimiter=',')
data.shape # 699 Samples given, 11 attributes per sample


(699, 11)

In [21]:
data[~np.isnan(data).any(axis=1)]
data.shape

AxisError: axis 1 is out of bounds for array of dimension 1